In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-09-30 18:50:09.929114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 18:50:09.929159: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
ds_name = 'mnli'
num_labels = 3
valid_split = 'validation_matched'
test_split = 'validation_mismatched'
runs = 5

db_path = "./mnli_9-29.db"
seed = 0
pool_size = 2000
search_size = 200
warmup_runs = 100
annealing_runs = 2000
wandb_project = 'mnli_search'
wandb_entity = 'johntzwei'

model_card = "bert-base-cased"
pretraining = True
max_steps = 3000
eval_steps = 500
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
training_args = select.SubsetTrainingArguments(num_labels=num_labels,
                                               model_card=model_card,
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(db_path=db_path,
                                                seed=seed, 
                                                data_pool_size=pool_size,
                                                optimal_subset_size=search_size,
                                                warmup_runs=warmup_runs,
                                                annealing_runs=annealing_runs,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [5]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)

Reusing dataset glue (/home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e3a2a05825463f25.arrow


  0%|          | 0/9815 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-042a9e4d87492384.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ed69df6abda2cd04.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-2ea3951d0a232e7b.arrow


In [6]:
processed_ds

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9847
    })
})

In [7]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds[valid_split], test_ds=processed_ds[test_split]
)

In [8]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(db_path)
df = df.sort_values('objective')
idx = json.loads(df.iloc[-1]['indexes'])

In [9]:
print(idx)
df

[668, 812, 532, 1755, 1013, 1327, 724, 1235, 662, 1782, 232, 1675, 772, 1056, 494, 897, 1572, 1980, 1225, 1489, 1131, 1764, 1884, 1366, 1275, 1114, 802, 676, 435, 73, 1753, 84, 1265, 1961, 328, 924, 64, 1038, 1278, 619, 1554, 191, 1610, 666, 1354, 514, 696, 1460, 604, 647, 723, 222, 1643, 1033, 37, 648, 1628, 471, 90, 890, 155, 1868, 1034, 1871, 1268, 425, 1923, 1765, 677, 303, 575, 1311, 394, 1314, 1174, 1239, 1692, 1285, 899, 1838, 1104, 1170, 1203, 1, 1645, 964, 596, 943, 228, 1888, 236, 1863, 744, 1844, 1066, 1192, 769, 1614, 937, 391, 1017, 1462, 347, 956, 436, 476, 1455, 665, 768, 1771, 1326, 1450, 1074, 242, 459, 1280, 1594, 1211, 1609, 1356, 903, 734, 1886, 1666, 1461, 440, 752, 1226, 840, 974, 1431, 885, 1847, 935, 1204, 1369, 874, 323, 496, 1353, 872, 1439, 572, 392, 386, 557, 1128, 811, 1772, 1040, 1535, 1120, 396, 1427, 620, 574, 294, 103, 1726, 31, 1544, 1164, 1522, 1075, 1108, 123, 271, 927, 656, 1859, 297, 40, 1419, 1853, 1873, 1107, 1612, 196, 1501, 1814, 1917, 925, 767

,indexes,objective
0,"[115, 421, 1639, 1862, 851, 707, 12, 1351, 129...",0.000000
55,"[616, 907, 991, 562, 1816, 1976, 1492, 1911, 1...",0.371676
38,"[569, 568, 1438, 732, 1240, 1071, 1364, 1872, ...",0.373714
31,"[957, 531, 117, 261, 1721, 1362, 1246, 733, 14...",0.376566
27,"[957, 1275, 1412, 638, 1630, 1566, 341, 1253, ...",0.378808
...,...,...
3615,"[668, 812, 532, 1755, 1013, 1327, 724, 1235, 6...",0.547937
3608,"[668, 812, 532, 1755, 1013, 1327, 724, 1235, 6...",0.547937
3635,"[668, 812, 532, 1755, 1013, 1327, 724, 1235, 6...",0.548141
3593,"[668, 812, 532, 1755, 1013, 1327, 724, 1235, 6...",0.548242


In [10]:
select_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    select_quality.append(quality)

wandb: Currently logged in as: johntzwei (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-09-30 18:50:25.152933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 18:50:25.152973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/3000 [00:00<?, ?it/s]

...

wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-09-30 19:39:41.983732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 19:39:41.983774: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/1227 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
np.mean(select_quality), np.std(select_quality)

(0.5377135882831571, 0.02734707864278273)

In [14]:
random_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    idx = np.random.choice(np.arange(pool_size), size=search_size)
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    random_quality.append(quality)
random_quality

loss,███▇█▇▆▄▃▆▂▂▄▂▂▂▁▅▁▄▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁
sst2_test:accuracy,▁
sst:val_acc,▁▆█▇██
loss,0.00086
sst2_test:accuracy,0.61361
sst:val_acc,0.5836


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-09-30 21:31:08.465013: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 21:31:08.465053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/3000 [00:00<?, ?it/s]

...

  0%|          | 0/1227 [00:00<?, ?it/s]

  0%|          | 0/1227 [00:00<?, ?it/s]

  0%|          | 0/1227 [00:00<?, ?it/s]

  0%|          | 0/1229 [00:00<?, ?it/s]

[0.43216029292107405,
 0.455553295362083,
 0.4652156224572823,
 0.40703824247355574,
 0.463079739625712]

In [15]:
np.mean(random_quality), np.std(random_quality)

(0.44460943856794144, 0.022145477373388858)